# Tutorial 2 - Feature exploration and selection (using cross-validation)
In this second tutorial, we will cover two main topics:
1. **Feature exploration**: In Tutorial 1, we had started to explore our data. In this tutorial, we will explore the data further, in order to design a feature set which enables better predictions.
2. **Feature selection**: Following the feature exploration, we will see how we can select features to improve the model performance. For this, we will use a default k-Nearest Neighbour algorithm (which we will assess further in Tutorial 3), which we train using k-fold cross-validation to reduce the randomnes in the train-test-split that we have used in Tutorial 1.

# Part 1: Data revisited
Let's look at the data again: In Tutorial 1, we looked at the statistics and the distribution of the individual variables. This is OK for an initial model, but in order to improve it, we also need to look at the **dependencies** between the variables. There are two types of dependencies that we are interested in:
- dependencies **between features** --> these dependencies are *__unwanted__*
- dependencies between **features and target** --> these dependencies are *__wanted__*

Why do we separate between the two? The two dependencies have different effects on the model: If there are a lot of *dependent features*, this means that these do not contribute to improving the model, since they do not provide *new* information. **A well-constructed feature set hence consists of independent or nearly independent features**.

On the other hand, if some features are independent of the target, this means that the data in these features does not contain any useful information for the prediction of the target. This can "confuse" a Machine Learning model (some more than others, as we will see), and hence *reduce performance*. **A well-constructed feature set hence shows a significant dependency between the target and each of the features.**

## Step 1.1 - Load packages and data
As in the previous tutorials, we start by loading the packages and data:

In [ ]:
# Import the necessary packages to read the data
import pandas as pd
import numpy  as np # numpy is always useful
import os           # Loads the functions related to the operating system 

In [ ]:
# TASK: Load the data from the csv file (with DF_UID as index column)

## Step 1.2 - Correlation between variables
The probably most widely used meausure of dependency between variables is their **correlation**. The standard (Pearson's) correlation coefficient between two variables $X$ and $Y$ is computed as:

$$ corr(X,Y) = \frac{cov(X,Y)}{\sigma_X \sigma_Y}$$

where $\sigma$ denotes the standard deviation. The maximum value of the correlation coefficient is 1, denoting a perfect linear dependency between $X$ and $Y$, and the minimum values is -1, denoting an exact inverse dependency of $X$ and $Y$. A correlation of 0 means that $X$ and $Y$ are *independent* of each other.

**NOTE**: From the above logic it follows that the correlation coefficient between a variable and itself is always 1.

Using `pandas`, we can compute the correlation between each pair of variables. To improve the readability of the resulting table, we will style the table with red colors for positive values, and blue colors for negative values:

In [ ]:
# Compute the correlation between variables
correlation_matrix = data.corr()
correlation_matrix.style.background_gradient(cmap='coolwarm', vmin = -1, vmax = 1).format("{:.2}") # Formatting

## *Questions*
- a. The corrlation matrix shows a strong negative correlation between the y-coordinate of the roof (roof_y) and SIS/SISDIR. Can you explain why? Why do we see no such correlation for roof_x?
- b. The correlation matrix also shows two correlated "blocks" of the horizon values. What causes these blocks and what does that imply for the model?
- c. Which are the five variables with the strongest correlation to the target variable? Which are the five variables with the smallest correlation?

## Step 1.3 - Visualising features vs. targets 
Visualising the relationship between variables and between the variables and the target can help to significantly improve our understanding of the data. In particular, it can help to identify non-linear relationships that are not captured by the correlation coefficient.

Pandas has a built-in function to visualise *all* relationships between features and the target as scatterplot, just like the correlation matrix. It is called as: `pd.plotting.scatter_matrix(data)`

As we have a lot of features and targets, this results in a huge plot that becomes difficult to interpret (but feel free to try it!). Here, we will look only at the relationships between each feature and the target.

To start, we set up the matplotlib interactive plotting:

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

and define the list of features and the target, to know what to plot against each other:

In [ ]:
target_name   = ## TASK: ADD THE NAME OF THE TARGET (from Tutorial 1)
features_list = ## TASK: ADD THE LIST OF FEATURES (from Tutorial 1)

Now, we can create scatterplots of all features against the target:

In [ ]:
# Create figure
fig, axes = plt.subplots(5,4, sharey = True, figsize = (10,10))
ax = axes.flatten()
for i, feature in enumerate(features_list):
    data.plot.scatter(x = feature, y = target_name, ax = ax[i], s = 2) # Plot feature against target
    # Format figure:
    ax[i].set_xlabel('')
    ax[i].set_title(feature)
    ax[i].grid(ls = '--')
    
    # For the roof area only, show the x-axis with logarithmic values:
    if feature == 'roof_area': ax[i].set_xscale('log')
for j in range(i+1, len(ax)): ax[j].axis('off') # deactivate unused sub-axes
plt.tight_layout()

## *Questions*
- a. Based on the scatterplots, which variable shows the strongest relatioship with the target?
- b. What is the correlation coefficient between this varialbe and the target?
- c. How do you explain the discrepancy, and what does this mean for the modelling?
- d. Can you think of a way to transform this variable in order to increase its correlation coefficient?
- e. For the other 4 features with the smallest correlation to the target from Step 1.2, do you observe a similar discrepancy or are the correlation coefficients confirmed by the plots?

## Step 1.4 - Feature transformation
Many algorithms, such as linear regression or KNN, perform better with features that are have a *linear* relationship with the target. As we saw in Step 1.3, this is not the case for the roof aspect, which however appears to be one of the most important features following the scatterplot. An inspection of the roof aspect actually hints to a very simple way to turn this feature into a linear one, by dividing it into two features without any loss of information:
- 1. The absolute value (computed using python's `abs()`-function)
- 2. The sign (computed using numpy's `np.sign()` function)

Let's transform the data in this way by computing first the sign and then replacing the roof aspect by it's absolute value:

In [ ]:
data['aspect_sign'] = ## TASK: COMPUTE THE SIGN OF roof_aspect
data['roof_aspect'] = ## TASK: COMPUTE THE ABSOLUTE VALUE OF roof_aspect

We can check the correlation coefficient of the new features with the tilted radiation by using the `corrwith` function, and will see that it is much improved and that the roof_aspect becomes the *most important feature*:

In [ ]:
data[['roof_aspect', 'aspect_sign']].corrwith(data['tilted_radiation'])

Finally, we need to add the new feature `aspect_sign` to the feature list:

In [ ]:
features_list = # TASK: Define new feature list that contains also aspect_sign

## Step 1.5 - Mutual information
The above look at the correlations has shown that simply looking at the correlation coefficient is not sufficient to represent the dependency between the features and the target. But it is also not the only way to look at dependencies: 

The **mutual information** is another measure of dependency, which quantifies the amount of information obtained on the target from observing a feature. It is based on the joint probabilistic distribution of the two variables, and can hence represent more complex dependencies.

We can compute the mutual information of the features towards the target using the `mutual_info_regression`from `sklearn.feature_selection`:

In [ ]:
from sklearn.feature_selection import mutual_info_regression

By splitting the data into target (y) and features (X), we can call the function and display it as a pandas dataframe:

In [ ]:
y = ## TASK: EXTRACT THE TARGET COLUMN
X = ## TASK: EXTRACT THE FEATURES

In [ ]:
(pd.DataFrame(mutual_info_regression(X, y), index = X.columns, columns = ['Mutual_info'])
     .sort_values('Mutual_info', ascending = False))

## *Questions*
- a. Based on mutual information, which are the most important features?
- b. How does this answer change compared to the answers from Step 1.2?

# Part 2 - Feature preprocessing and k-fold cross-validation
In Part 1, we have looked again at the features, to develop an understanding of which features are important for the modelling. In step 2, we will pre-process the dataset, so that we can perform some feature selection and training in Step 3. 

In contrast to the previous tutorials, we will now use **cross-validation** instead of simple validation, and hence no longer need to divide the data into training and validation set. To see how the k-fold cross-validation works. In addition, we need to *scale* the data.

The required packages are:

In [ ]:
from sklearn.preprocessing   import scale
from sklearn.model_selection import KFold

from lib_file import *    # Functions in the file lib_file.py, which should be in the same directory as your notebook

## Step 2.1: k-fold cross-validation
To understand how the k-fold cross-validation splits the data, we can declare a 5-fold cross-validation, which is the underlying "building block" of all `scikit-learn`-functions that perform cross-validation. The aim here is only to show how it works; in part 3, we will the cross-validation in model fitting and scoring.

Throughout this tutorial we will work with 5-fold cross-validation, which is the default in all of scikit's `cross_val`-functions:

In [3]:
N_FOLDS = 5 # Define the number of folds

We can declare a 5-fold cross-validation and plot the resulting splits for our data:

In [ ]:
cv = KFold(n_splits = N_FOLDS)
cv

In [ ]:
plt.figure(figsize = (7, 3))
plot_cv_indices(cv, X, y, ax = plt.gca(), n_splits = N_FOLDS, cmap_cv = 'coolwarm')
plt.tight_layout()

As you can see in the figure, the k-fold cross-validation always selects **consecutive** parts of the 1000 data samples for each of the folds. This can pose a problem if the data is *sorted*, as it may lead to large differences between folds.

## Step 2.2: Prepare features and target
To avoid having always the same (and potentially biased) subsets of the data in the cross-validation, let's shuffle the entire dataset randomly:

In [ ]:
data_shuffled = data.sample( frac = 1 ) # Randomly sample from the entire dataset (i.e. reshuffle the data)

**NOTE**: This step introduces additional randomness in the modelling, and means that your results will vary every time you run the script.

Now, we need to extract the features and targets again, from the shuffled dataset:

In [ ]:
y = ## TASK: EXTRACT THE TARGET COLUMN FROM data_shuffled
X = ## TASK: EXTRACT THE FEATURES FROM data_shuffled

Finally, we should standardize the data using the `scale` function:

In [ ]:
X_scaled = scale(X)

Later, we will need to know the total number of features, so let's assign this value to the variable `n_features_all`:

In [ ]:
n_features_all = X_scaled.shape[1]
n_features_all

## Step 2.3: Save new datasets
To avoid having to re-do the feature transformation from Step 1.4 and the feature preprocessing steps in Step 2.2, we can save the new shuffled and scaled datasets for future reference.

To save the shuffled (and transformed) data, we can simply save the varible `data_shuffled` to a .csv file:

In [ ]:
data_shuffled.to_csv('training_shuffled.csv')

To save the scaled data, we first need to convert the *array* `X_scaled` into a dataframe (to attach the correct index and column names) and concatenate it with the target. We can do this as:

In [ ]:
data_scaled = pd.concat([ pd.DataFrame(X_scaled, columns = X.columns, index = X.index), # Dataframe for X_scaled
                          y ], axis = 1)                                                # Append y horizontally

In [ ]:
# TASK: Save the variable "data_scaled" to the csv file training_scaled.csv

# Part 3 - Feature selection using KNN
To perform feature selection, we will use the KNN (k-nearest neighbour algorithm), since it is a very quick algorithm that performs well even with the default parameters (in tutorial 3, we will look deeper into the KNN's hyper-parameters and their tuning). Using KNN for feature selection will help us to find an answer to the question:

_**Which features should we include in the model?**_

The analysis of the features has shown that we have some strong dependencies, so we probably do not want to include *all* features. But which are the best ones to include? To try to find an answer, we can simply select the _K_ best features, whereby we will use the mutual information to define what is "best". In Steps 3.1 and 3.2, we will evaluate the optimal value for _K_.

For the feature selection using KNN the following functions are needed:

In [ ]:
from sklearn.feature_selection import SelectKBest          # The feature selection module
from sklearn.neighbors         import KNeighborsRegressor  # The KNN regression model
from sklearn.model_selection   import cross_val_score      # The cross-validation function

## Step 3.1: Feature selection - a first glance
To perform feature selection, we will use the `mutual_info_regression`-function to define *which* are the best features to select. Let's select, just as a test, half of the features. We can obtain the 8 best features using the `SelectKBest`-function such as:

In [ ]:
# Set up a SelectKBest instance with mutual information as decision criterion to return the 8 best features
mutual_info = SelectKBest(mutual_info_regression, k = 8 )          # Initialise
X_reduced   = mutual_info.fit_transform(X_scaled, y)               # Extract the best features

Then, we can declare a `KNeighborsRegressor` and perform a cross-validation for the reduced dataset. This requires to pass the model, the features and the target to `cross_val_score`.

**NOTE**: The `fit` and `predict`-procedures we have seen in Tutorial 1 are applied *inside* the `cross_val_score` to the passed estimator for each fold. *Fit* and *predict/score* always form the basis of any Machine Learning task.

In [ ]:
knn      = KNeighborsRegressor() # Initiate model
cv_score = cross_val_score(knn, X_reduced, y, cv = N_FOLDS)
cv_score

The `cv_score` shows the results for all folds, but often it is more informative to obtain the *mean* and *standard deviation* instead:

In [ ]:
print( cv_score.mean() )
print( cv_score.std() )

##### Comparison with all features
As a comparison, let's look at the score obtained from the default knn with *all* features:

In [ ]:
knn      = # TASK: Initiate default KNN
cv_score = # TASK: Obtain the cross-validation score with all data

In [ ]:
# TASK: Display mean cv score
# TASK: Display std. deviation of cv score

##### Comparison with default feature set (before feature transformation)
Another comparison that we can perform is to the default dataset, i.e. before applying the feature transformation from Step 1.4. We have wrapped this analysis in the function `cross_val_score_default_knn()` (found in the lib_file.py). The default CV score (all features, no transformation) is:

In [ ]:
cv_score = cross_val_score_default_knn()
print( cv_score.mean() )
print( cv_score.std() )

## *Questions*
- a. Which performance metric is used in the `cross_val_score` method? What does a score of 1 represent? What about a score of 0?
- b. How is the performance of the model with the default feature set improved through feature transformation (with all features) and subsequently feature selection (keeping 8 features)?
- c. What does this very first glance suggest about the importance of feature engineering (transformation, selection, etc.)?

## Step 3.2: Systematic feature selection
Now, let's approach the model selection in a more systematic way, but iterating over all possible number of features.

**NOTE**: We will do this step using the default knn model in this tutorial. However, be aware that the optimal number of features can **change** for different models and for different hyper-parameters!

In [ ]:
n_features = np.arange(1, n_features_all + 1) # Construct a list of all possible number of features
n_features

We next declare an empty dataframe that contains the k folds as rows, and the different number of features as columns:

In [ ]:
cv_featureSelect = pd.DataFrame(index = range(N_FOLDS), columns = n_features).fillna(0)
cv_featureSelect.index.name   = 'CV iteration'
cv_featureSelect.columns.name = 'Number of features'

By iterating over all possible number of features, we compute the cross-validation scores for the default knn:

In [ ]:
# Iterate over all possible numbers of features
for n in n_features: 
    # Extract the n best features
    mutual_info = SelectKBest(mutual_info_regression, k = n )
    X_reduced   = mutual_info.fit_transform(X_scaled, y)
    
    # Train a knn and score the performance with 5-fold cross-validation
    knn = KNeighborsRegressor() # Initiate model
    cv_featureSelect.loc[:,n] = cross_val_score(knn, X_reduced, y, cv = N_FOLDS)   

This is the resulting dataframe:

In [ ]:
# Display the data
cv_featureSelect.round(3)

**NOTE**: Since the data is randomly shuffled before splitting, the results will deviate between different runs and from the solution!

Let's compute, for each number of features, the mean and standard deviation, which we can visualise as a pandas dataframe:

In [ ]:
# Obtain cross-validation statistics
cv_stats = pd.DataFrame( [ cv_featureSelect.mean().rename('mean'),    # compute mean across the k folds
                           cv_featureSelect.std() .rename('std') ] )  # compute std. deviaiton across the k folds

In [ ]:
# Visualise dataframe with formatting
format_df( cv_stats, cmap='YlGn', decimals = 3)

**NOTE**: For simplicity and readability of the code, we have created a function `format_df` in the lib_file.py, which allows to display tables with colored background.

The performance of the KNN can also be visualised as a plot with errorbars:

In [ ]:
plt.figure(figsize = (8,3))
cv_stats.loc['mean'].plot(yerr = cv_stats.loc['std'], ax = plt.gca())
plt.ylabel('Cross-validation score ($R^2$)')
plt.grid()
plt.tight_layout()

## *Questions*
- a. What is the optimal number of features? **NOTE**: These results can vary due to the re-shuffling of the data!
- b. How much of the improvement compared to the default model from tutorial 2 can be attributed to the feature transformation of the `roof_aspect`? How do you measure that?
- c. How much of the improvement can be attributed to the feature selection?